In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [4]:
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test.csv
Saving heart_train.csv to heart_train.csv


In [0]:
train_file_path = "heart_train.csv"
test_file_path = "heart_test.csv"

In [0]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [7]:
!head {train_file_path}

row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,160,12,5.73,23.11,Present,49,25.3,97.2,52,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
4,170,7.5,6.41,38.03,Present,51,31.99,24.26,58,1
5,134,13.6,3.5,27.78,Present,60,25.99,57.34,49,1
6,132,6.2,6.47,36.21,Present,62,30.77,14.14,45,0
7,142,4.05,3.38,16.2,Absent,59,20.81,2.62,38,0
8,114,4.08,4.59,14.6,Present,62,23.11,6.72,58,1
9,114,0,3.83,19.4,Present,49,24.86,2.49,29,0


In [0]:
# The only column you need to identify explicitly is the one with the value that the model is intended to predict.
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

In [9]:
# read the CSV data from the file and create two datasets for training and testing

def get_dataset(file, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file,
      batch_size = 50,
      label_name=LABEL_COLUMN,
      na_value ="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

info = ['sbp','tobacco','ldl','adiposity','famhist', 'typea','obesity','alcohol','age','chd']
raw_train_data = get_dataset(train_file_path, select_columns=info)
raw_test_data = get_dataset(test_file_path, select_columns=info)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [11]:
show_batch(raw_train_data)


sbp                 : [128 134 118 144 170 138 122 168 134 114 129 120 124 128 162 136 114 140
 158 116 152 117 112 143 122 148 118 136 101 170 164 132 114 106 128 110
 132 122 126 114 128 154 144 180 120 138 162 128 140 134]
tobacco             : [ 2.6   1.5   0.75  0.04  2.6   1.15  0.    9.    0.9   0.1   2.15  1.4
  0.    0.5   5.3  11.2   4.08  8.   13.5   1.91 12.18  1.53  9.65  0.46
  3.2   0.    0.12  3.46  0.48  0.    0.5  12.3   0.    5.6   0.7   2.35
  0.72  0.    8.75  1.2   0.    4.5   2.4   3.57 10.5   0.    5.6   1.6
  3.9   3.  ]
ldl                 : [4.94 3.73 2.58 3.38 7.22 5.09 3.76 8.53 3.18 3.95 5.17 6.25 2.28 3.7
 7.95 5.81 4.59 4.42 5.04 7.56 4.04 2.44 2.29 2.4  3.59 4.66 3.26 6.38
 7.26 3.12 6.95 5.96 4.97 3.2  4.9  3.36 4.37 5.49 6.53 3.98 6.34 4.75
 8.13 3.57 2.7  2.68 4.24 5.41 7.32 3.17]
adiposity           : [21.36 21.53 20.25 23.61 28.69 27.87 24.59 24.48 23.66 15.89 27.57 20.47
 24.86 12.81 33.58 31.85 14.6  33.15 30.79 26.45 37.83 28.95 17.2  22.87
 22.

In [0]:
test_batch, labels_batch = next(iter(raw_test_data))
train_batch, labels_batch = next(iter(raw_train_data)) 

In [0]:
# simple function that packs together all the columns
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [0]:
# define a more general preprocessor that selects a list of numeric features and packs them into a single column:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['sbp','tobacco','ldl','adiposity', 'typea','obesity','alcohol','age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [16]:
show_batch(packed_train_data)

famhist             : [b'Absent' b'Present' b'Present' b'Present' b'Absent' b'Absent' b'Absent'
 b'Present' b'Absent' b'Absent' b'Absent' b'Absent' b'Present' b'Absent'
 b'Absent' b'Present' b'Present' b'Absent' b'Present' b'Absent' b'Present'
 b'Absent' b'Absent' b'Absent' b'Absent' b'Present' b'Absent' b'Absent'
 b'Absent' b'Present' b'Absent' b'Absent' b'Present' b'Absent' b'Absent'
 b'Present' b'Absent' b'Absent' b'Absent' b'Present' b'Present' b'Absent'
 b'Absent' b'Present' b'Absent' b'Present' b'Absent' b'Present' b'Present'
 b'Present']
numeric             : [[108.     0.     2.74  11.17  53.    22.61   0.95  20.  ]
 [118.     0.08   3.48  32.28  52.    29.14   3.81  46.  ]
 [158.     2.6    7.46  34.07  61.    29.3   53.28  62.  ]
 [132.     2.     2.7   21.57  50.    27.95   9.26  37.  ]
 [120.     1.4    6.25  20.47  60.    25.85   8.51  28.  ]
 [118.     0.     3.67  12.13  51.    19.15   0.6   15.  ]
 [118.     0.     2.39  12.13  49.    18.46   0.26  17.  ]
 [162.     7.4

In [0]:
train_batch, label_batch = next(iter(packed_train_data))
test_batch, label_batch = next(iter(packed_test_data))

In [18]:
# data normalization

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,137.484925,3.669020,4.776055,25.428291,53.386935,26.145804,16.936206,43.198492
std,19.385614,4.490931,2.025025,7.664637,9.954609,4.222120,24.642372,14.410089
min,101.000000,0.000000,0.980000,7.120000,13.000000,17.750000,0.000000,15.000000
25%,124.000000,0.100000,3.272500,19.805000,47.000000,23.222500,0.510000,32.000000
50%,134.000000,2.085000,4.400000,26.210000,54.000000,25.830000,7.710000,45.000000
75%,148.000000,5.575000,5.900000,31.212500,60.000000,28.470000,23.395000,56.000000
max,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [21]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(8,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f8874568d08>, mean=array([137.485,   3.669,   4.776,  25.428,  53.387,  26.146,  16.936,
        43.198]), std=array([19.386,  4.491,  2.025,  7.665,  9.955,  4.222, 24.642, 14.41 ])))

In [22]:
train_batch['numeric']

<tf.Tensor: id=266, shape=(50, 8), dtype=float32, numpy=
array([[154.  ,   0.  ,   4.81,  28.11,  56.  ,  25.67,  75.77,  59.  ],
       [142.  ,   0.28,   1.8 ,  21.03,  57.  ,  23.65,   2.93,  33.  ],
       [118.  ,   0.  ,   2.39,  12.13,  49.  ,  18.46,   0.26,  17.  ],
       [120.  ,   0.  ,   3.68,  12.24,  51.  ,  20.52,   0.51,  20.  ],
       [118.  ,   1.5 ,   5.38,  25.84,  64.  ,  28.63,   3.89,  29.  ],
       [112.  ,   1.44,   2.71,  22.92,  59.  ,  24.81,   0.  ,  52.  ],
       [122.  ,   1.  ,   5.88,  34.81,  69.  ,  31.27,  15.94,  40.  ],
       [116.  ,   4.28,   7.02,  19.99,  68.  ,  23.31,   0.  ,  52.  ],
       [114.  ,   1.2 ,   3.98,  14.9 ,  49.  ,  23.79,  25.82,  26.  ],
       [143.  ,   5.04,   4.86,  23.59,  58.  ,  24.69,  18.72,  42.  ],
       [136.  ,   1.36,   3.16,  14.97,  56.  ,  24.98,   7.3 ,  24.  ],
       [164.  ,  13.02,   6.26,  29.38,  47.  ,  22.75,  37.03,  54.  ],
       [136.  ,   0.  ,   2.28,  18.14,  55.  ,  22.59,   0.  ,  17

In [23]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

array([[ 0.852, -0.817,  0.017,  0.35 ,  0.262, -0.113,  2.388,  1.097],
       [ 0.233, -0.755, -1.47 , -0.574,  0.363, -0.591, -0.568, -0.708],
       [-1.005, -0.817, -1.178, -1.735, -0.441, -1.82 , -0.677, -1.818],
       [-0.902, -0.817, -0.541, -1.721, -0.24 , -1.332, -0.667, -1.61 ],
       [-1.005, -0.483,  0.298,  0.054,  1.066,  0.588, -0.529, -0.985],
       [-1.315, -0.496, -1.02 , -0.327,  0.564, -0.316, -0.687,  0.611],
       [-0.799, -0.594,  0.545,  1.224,  1.568,  1.214, -0.04 , -0.222],
       [-1.108,  0.136,  1.108, -0.71 ,  1.468, -0.672, -0.687,  0.611],
       [-1.211, -0.55 , -0.393, -1.374, -0.441, -0.558,  0.361, -1.194],
       [ 0.284,  0.305,  0.041, -0.24 ,  0.463, -0.345,  0.072, -0.083],
       [-0.077, -0.514, -0.798, -1.364,  0.262, -0.276, -0.391, -1.332],
       [ 1.368,  2.082,  0.733,  0.516, -0.642, -0.804,  0.815,  0.75 ],
       [-0.077, -0.817, -1.233, -0.951,  0.162, -0.842, -0.687, -1.818],
       [-0.18 , -0.483, -0.517, -0.509, -1.244, -0.

In [0]:
CATEGORIES = {
    'famhist': ['Absent', 'Present']
}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [26]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='famhist', vocabulary_list=('Absent', 'Present'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [28]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(train_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1.]


In [0]:
#Combined preprocessing layer
#Add the two feature column collections and pass them to a tf.keras.layers.DenseFeatures to create an input layer that will extract and preprocess both input types:

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [31]:
print(preprocessing_layer(train_batch).numpy()[0])

[ 0.     1.     0.852 -0.817  0.017  0.35   0.262 -0.113  2.388  1.097]


In [0]:
# build the model

model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  tf.keras.layers.Dropout(0.2),

  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adamax',
    metrics=['accuracy'])


In [0]:
train_data = packed_train_data.shuffle(600)
test_data = packed_test_data

In [155]:
model.fit(train_data, epochs=400)

Epoch 1/400
8/8 [==============================] - 1s 74ms/step - loss: 0.6484 - accuracy: 0.6407
Epoch 2/400
8/8 [==============================] - 0s 7ms/step - loss: 0.6055 - accuracy: 0.6910
Epoch 3/400
8/8 [==============================] - 0s 7ms/step - loss: 0.5687 - accuracy: 0.7060
Epoch 4/400
8/8 [==============================] - 0s 10ms/step - loss: 0.5421 - accuracy: 0.7211
Epoch 5/400
8/8 [==============================] - 0s 9ms/step - loss: 0.5332 - accuracy: 0.7337
Epoch 6/400
8/8 [==============================] - 0s 8ms/step - loss: 0.5310 - accuracy: 0.7337
Epoch 7/400
8/8 [==============================] - 0s 9ms/step - loss: 0.5286 - accuracy: 0.7412
Epoch 8/400
8/8 [==============================] - 0s 8ms/step - loss: 0.5226 - accuracy: 0.7337
Epoch 9/400
8/8 [==============================] - 0s 8ms/step - loss: 0.5128 - accuracy: 0.7462
Epoch 10/400
8/8 [==============================] - 0s 9ms/step - loss: 0.4886 - accuracy: 0.7513
Epoch 11/400
8/8 [=========

In [153]:
train_loss, train_accuracy = model.evaluate(train_data)

print('\n\nTrain Loss {:.4f}, Train Accuracy {:.2%}'.format(train_loss, train_accuracy))

8/8 [==============================] - 0s 9ms/step - loss: 0.1436 - accuracy: 0.9698


Train Loss 0.1436, Train Accuracy 96.98%


In [156]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {:.4f}, Test Accuracy {:.2%}'.format(test_loss, test_accuracy))

2/2 [==============================] - 0s 73ms/step - loss: 0.5474 - accuracy: 0.7969


Test Loss 0.5474, Test Accuracy 79.69%


In [97]:
predictions = model.predict(test_data)

# Show some results
for prediction, CHD in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted diagnosis certainty: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("Patient has CHD" if bool(CHD) else "Patient doesn't have CHD"))


Predicted diagnosis certainty: 0.00%  | Actual outcome:  Patient doesn't have CHD
Predicted diagnosis certainty: 2.46%  | Actual outcome:  Patient has CHD
Predicted diagnosis certainty: 91.62%  | Actual outcome:  Patient doesn't have CHD
Predicted diagnosis certainty: 3.35%  | Actual outcome:  Patient has CHD
Predicted diagnosis certainty: 99.16%  | Actual outcome:  Patient doesn't have CHD
Predicted diagnosis certainty: 0.03%  | Actual outcome:  Patient doesn't have CHD
Predicted diagnosis certainty: 95.79%  | Actual outcome:  Patient has CHD
Predicted diagnosis certainty: 0.54%  | Actual outcome:  Patient has CHD
Predicted diagnosis certainty: 99.98%  | Actual outcome:  Patient doesn't have CHD
Predicted diagnosis certainty: 0.21%  | Actual outcome:  Patient doesn't have CHD
